Intro requests for review.  Programming is a collaborative and creative process that brings ideas to life through the development of software. In the Create performance task, you will design and implement a program that solves a problem, enables innovation, explores personal interests, or expresses creativity.
-Purpose of your groups program
-Purpose of your individual feature(s)
Input/Output requests.  Demo ways to Input to your full stack feature.
-Using frontend show  API request and  present API response. (live demo)
-Using postman to show raw API request and RESTful response (error code(s) and JSON)
-Using db_init, db_restore, db_backup to show tester data creation and data recovery.
List requests.  Use of list, dictionaries and database.  Code descriptions of area where you work with list (rows) and dictionaries (columns) of the database.
-Discuss formatting response data (JSON) from API into DOM
-Discuss queries from database where you extract a Python List (rows). Mention how these queries are provide by a 3rd. party library.
-Discuss methods in "class"  you created to work with columns (create, read, update, delete)
Algorithmic code request. Show the definition of code blocks to handle a request.
-Discuss API class (code block) you used to perform  get, post, put, and delete methods.
-Discuss a method/procedure in class that contains sequencing, selection, and iteration.
-Discuss the parameters (body of request) and return type (jasonify) of the function.
Call to Algorithm request.  Show the definition of code block to make a request.
-Discuss the call/request to the method with Algorithm (fetch to endpoint).
-Discuss the return/response from the method with Algorithm (fetch) and how you handle data.
-Show how changing data or method triggers a different response, specifically normal conditions and error conditions.

# Sprint 5 Full Stack and CRUD Review

## Introduction
Our group decided to make a book review site where users can talk about and interact with books while exploring new suggestions for what to read. We chose this for our group project due to our shared interest in reading.
<br>
<br>
My individual feature is a random book generator where users get random suggestions for what to read next base don the genre they selected. When a user inputs a genre, the random suggestion they recieve is displayed with the book's author, title, short summary, and book cover image. I'm currently working on adding a sub-feature where users can add more books (as suggestions) into the random book generator so other users can get a book reccomendation from a bigger database. I haven't connected this "adding book reccomendations" feature to the main database and it has it's own table in backend. 



## Frontend Demo for Full Stack

This is demonstration of a user adding a new book reccomendation to my feature through the frontend. This is what the page looks like at first:
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/bookrec_start_page.png">
<br>
The form to add a new book appears when the user clicks the "Add a New Book Reccomendation" button:
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/add_bookrec_empty_form.png">
<br>
This is the filled out form (Each input box is required):
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/filled_bookrec_form.png">
<br>
Once the user has filled out the form and has clicked the "Done" button, a console message appears to confirm that their form has been added to the backend table:
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/bookrec_form_done.png">
<br>
After clicking "Done", the new book appears in the backend table (number 3):
<br>
<img src="{{site.baseurl}}/images/notebooks/sprint_5_live_review/backend_bookrec_added.png">
<br>

In [ ]:
#The frontend demo above connects to this section of the bookrec API:
@bookrec_api.route("/add_bookrec", methods=['POST']) # This is the endpoint to add a book to the savebookrec table
def add_book():
    data = request.get_json() # Get the data from the request
    title = data.get('title') 
    author = data.get('author')
    genre = data.get('genre')
    description = data.get('description')
    cover_image_url = data.get('cover_image_url') 

    if not title or not author:
        return jsonify({"error": "Title and author are required"}), 400

    new_book = SaveBookRec( # Create a new book object
        title=title,
        author=author,
        genre=genre,
        description=description,
        cover_image_url=cover_image_url
    )

    db.session.add(new_book) # Add the new book to the savebookrec table
    db.session.commit() # Commit the changes to the database
    
    return jsonify({"message": "Book added successfully"}), 201